In [3]:
import pandas as pd

ldeo_pco2_file_path = 'STSData/LDEO_Database_V2019.csv'

pco2_df = pd.read_csv(ldeo_pco2_file_path, dtype=str)

KeyboardInterrupt: 

In [ ]:
pco2_df.head()

In [ ]:
pco2_df.shape[0]

In [ ]:
pco2_df['YEAR'] = pco2_df['MONTH/DAY/YEAR'].str.split('/').str[2]
pco2_df['YEAR'] = pd.to_numeric(pco2_df['YEAR'], errors='raise')
pco2_df['MONTH'] = pco2_df['MONTH/DAY/YEAR'].str.split('/').str[0]
pco2_df['MONTH'] = pd.to_numeric(pco2_df['MONTH'], errors='raise')
pco2_df['DAY'] = pco2_df['MONTH/DAY/YEAR'].str.split('/').str[1]
pco2_df['DAY'] = pd.to_numeric(pco2_df['DAY'], errors='raise')
pco2_df['LAT'] = pd.to_numeric(pco2_df['LAT'], errors='raise')
pco2_df['LON'] = pd.to_numeric(pco2_df['LON'], errors='raise')
pco2_df['TEMP'] = pd.to_numeric(pco2_df['TEMP'], errors='raise')
pco2_df['SAL'] = pd.to_numeric(pco2_df['SAL'], errors='raise')
pco2_df['PCO2_SST'] = pd.to_numeric(pco2_df['PCO2_SST'], errors='raise')
pco2_df_filtered = pco2_df[pco2_df['YEAR'] >= 2011]
pco2_df_processed = pco2_df_filtered[['LAT','LON','TEMP','SAL','PCO2_SST','YEAR','MONTH','DAY']]

In [ ]:
pco2_df_processed.head()

In [ ]:
pco2_df_processed.shape[0]

In [7]:
def estimate_ta(lat, lon, sss, sst):
    if(((-110 <= lon <= -75 and -20 <= lat <= 20) or (-140 <= lon <= -110 and -10 <= lat <= 10)) and sst >= 18 and 31 <= sss <= 36.5):
        return 2294 + 64.88*(sss - 35) + 0.39*(sss - 35)**2 - 4.52*(sst - 29) - 0.232*(sst - 29)**2
    elif -30 <= lat <= 30 and sst >= 20 and 31 <= sss <= 38:
        return 2294 + 64.88*(sss - 35) + 0.39*(sss - 35)**2 - 4.52*(sst - 29) - 0.232*(sst - 29)**2
    elif 30 <= lat <= 80 and lon >= -100 and 0 <= sst <= 20 and 31 <= sss <= 37:
        return 2305 + 53.97*(sss - 35) + 2.74*(sss - 35)**2 - 1.16*(sst - 20) - 0.040*(sst - 20)**2	
    elif lat >= 30 and lon <= -100 and sst <= 20 and 31 <= sss <= 35:
        return 2305 + 53.23*(sss - 35) + 1.85*(sss - 35)**2 - 14.72*(sst - 20) - 0.158*(sst - 20)**2 + 0.062*(sst - 20)*lon
    elif -70 <= lat <= -30 and sst <= 20 and 33 <= sss <= 36:
        return 2305 + 52.48*(sss - 35) + 2.85*(sss - 35)**2 - 0.49*(sst - 20) + 0.086*(sst - 20)**2
    return -1

In [ ]:
from tqdm import tqdm
ta_values = []
for index, row in tqdm(pco2_df_processed.iterrows(), desc="Processing...", total=12906361, ncols=200,ascii=True):
    ta_values.append(estimate_ta(row.LAT, row.LON, row.SAL, row.TEMP))

In [10]:
import numpy as np

np_ta_values = np.array(ta_values)

np.save('Calculated_TA.npy', np_ta_values)

In [25]:
glodap_file_path = 'GLODAPv2.2023_Merged_Master_File.csv'

glodap_df = pd.read_csv(glodap_file_path, dtype=str)

In [26]:
glodap_df.shape[0]

1402829

In [27]:
glodap_df.head()

,G2expocode,G2cruise,G2station,G2region,G2cast,G2year,G2month,G2day,G2hour,G2minute,...,G2tocf,G2doc,G2docf,G2don,G2donf,G2tdn,G2tdnf,G2chla,G2chlaf,G2doi
0,06AQ19840719,1.0,319.0,4.0,1.0,1984.0,7.0,20.0,14.0,46.0,...,9.0,-9999,9.0,-9999,9.0,-9999,9.0,-9999,9.0,https://doi.org/10.3334/cdiac/otg.carina_06aq1...
1,06AQ19840719,1.0,319.0,4.0,1.0,1984.0,7.0,20.0,14.0,46.0,...,9.0,-9999,9.0,-9999,9.0,-9999,9.0,-9999,9.0,https://doi.org/10.3334/cdiac/otg.carina_06aq1...
2,06AQ19840719,1.0,319.0,4.0,1.0,1984.0,7.0,20.0,14.0,46.0,...,9.0,-9999,9.0,-9999,9.0,-9999,9.0,-9999,9.0,https://doi.org/10.3334/cdiac/otg.carina_06aq1...
3,06AQ19840719,1.0,319.0,4.0,1.0,1984.0,7.0,20.0,14.0,46.0,...,9.0,-9999,9.0,-9999,9.0,-9999,9.0,-9999,9.0,https://doi.org/10.3334/cdiac/otg.carina_06aq1...
4,06AQ19840719,1.0,319.0,4.0,1.0,1984.0,7.0,20.0,14.0,46.0,...,9.0,-9999,9.0,-9999,9.0,-9999,9.0,-9999,9.0,https://doi.org/10.3334/cdiac/otg.carina_06aq1...


In [32]:
import numpy as np
np_ph = glodap_df['G2phtsinsitutp'].to_numpy().astype(float)
np.count_nonzero(np_ph>=2)

490379

In [14]:
bad_count = np.count_nonzero(np_ta_values==-1)

In [15]:
np_pco2 = pco2_df_processed.to_numpy()

In [16]:
%%writefile estimation.py

def estimate_ta(lat, lon, sss, sst):
    if(((-110 <= lon <= -75 and -20 <= lat <= 20) or (-140 <= lon <= -110 and -10 <= lat <= 10)) and sst >= 18 and 31 <= sss <= 36.5):
        return 2294 + 64.88*(sss - 35) + 0.39*(sss - 35)**2 - 4.52*(sst - 29) - 0.232*(sst - 29)**2
    elif -30 <= lat <= 30 and sst >= 20 and 31 <= sss <= 38:
        return 2294 + 64.88*(sss - 35) + 0.39*(sss - 35)**2 - 4.52*(sst - 29) - 0.232*(sst - 29)**2
    elif 30 <= lat <= 80 and lon >= -100 and 0 <= sst <= 20 and 31 <= sss <= 37:
        return 2305 + 53.97*(sss - 35) + 2.74*(sss - 35)**2 - 1.16*(sst - 20) - 0.040*(sst - 20)**2	
    elif lat >= 30 and lon <= -100 and sst <= 20 and 31 <= sss <= 35:
        return 2305 + 53.23*(sss - 35) + 1.85*(sss - 35)**2 - 14.72*(sst - 20) - 0.158*(sst - 20)**2 + 0.062*(sst - 20)*lon
    elif -70 <= lat <= -30 and sst <= 20 and 33 <= sss <= 36:
        return 2305 + 52.48*(sss - 35) + 2.85*(sss - 35)**2 - 0.49*(sst - 20) + 0.086*(sst - 20)**2
    return -1

def wrapper(row):
    return estimate_ta(row[0], row[1], row[3], row[2])

Writing estimation.py


In [24]:
from tqdm import tqdm

from multiprocess import Pool
from estimation import wrapper

with Pool(12) as p:
    ta_values = list(
        tqdm(
            p.imap(wrapper,
                   np_pco2, chunksize=1000000),
            total=np_pco2.shape[0]
        )
    )    

100%|███████████████████████████████████████████████████████████████████| 12906361/12906361 [14:37<00:00, 14711.01it/s]
